# Exprimento 2 - AutoArima

Se usó autoarima para la agregación cliente_x_producto.
Es decir, cada <cliente, producto> será una serie.

PD: dió horrible.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../../data/preprocessed/base.csv", sep=",")
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
0,201701,10234,20524,0,2,0.05300,0.05300,HC,VAJILLA,Cristalino,Importado,500.0,NaN
1,201701,10032,20524,0,1,0.13628,0.13628,HC,VAJILLA,Cristalino,Importado,500.0,NaN
2,201701,10217,20524,0,1,0.03028,0.03028,HC,VAJILLA,Cristalino,Importado,500.0,NaN
3,201701,10125,20524,0,1,0.02271,0.02271,HC,VAJILLA,Cristalino,Importado,500.0,NaN
4,201701,10012,20524,0,11,1.54452,1.54452,HC,VAJILLA,Cristalino,Importado,500.0,NaN


In [3]:
productos_ok = pd.read_csv("../../data/raw/product_id_apredecir201912.csv")
productos_ok

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005
...,...
775,21263
776,21265
777,21266
778,21267


In [4]:
df = df[df['product_id'].isin(productos_ok['product_id'].unique())]
df.shape

(2293481, 13)

In [5]:
# 1. Agrupar las ventas por periodo, customer_id y product_id
df = df.groupby(['periodo', 'customer_id', 'product_id'])['tn'].sum().reset_index()

# 2. Convertir 'periodo' a datetime
df['periodo_dt'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')

# 3. Definir los 36 meses deseados (modificá fechas si tu dataset tiene otro rango)
todos_los_periodos = pd.date_range(start='2017-01-01', periods=36, freq='MS')
periodos_df = pd.DataFrame({'periodo_dt': todos_los_periodos})

# 4. Crear combinaciones cliente-producto
clientes_productos = df[['customer_id', 'product_id']].drop_duplicates()

# 5. Producto cartesiano entre periodos y combinaciones cliente-producto
completo = clientes_productos.merge(periodos_df, how='cross')  # requiere pandas >= 1.2.0

# 6. Merge con las ventas reales
df = df[['customer_id', 'product_id', 'periodo_dt', 'tn']]
df_completo = completo.merge(df, on=['customer_id', 'product_id', 'periodo_dt'], how='left')

# 7. Completar con ceros donde no hay venta
df_completo['tn'] = df_completo['tn'].fillna(0)

# 8. Agregar columna 'periodo' como int YYYYMM
df_completo['periodo'] = df_completo['periodo_dt'].dt.strftime('%Y%m').astype(int)

# Opcional: ordenar
df_completo = df_completo.sort_values(['customer_id', 'product_id', 'periodo'])

print(f"✅ Dataset final con series completas: {df_completo.shape[0]} filas")


✅ Dataset final con series completas: 9460980 filas


In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
from tqdm import tqdm

# Cargar dataset
df = df_completo.copy()
df["periodo_dt"] = pd.to_datetime(df["periodo"].astype(str), format="%Y%m")
df["unique_id"] = df["product_id"].astype(str) + "_" + df["customer_id"].astype(str)

df_forecast = df[["unique_id", "periodo_dt", "tn"]].rename(columns={
    "periodo_dt": "ds",
    "tn": "y"
})
df_forecast = df_forecast.sort_values(by=["unique_id", "ds"])

# Obtener todas las series únicas
all_series = df_forecast["unique_id"].unique()
chunk_size = 1000  # ajustar según memoria disponible
results = []

print("📈 Ejecutando AutoARIMA por bloques...\n")
for i in tqdm(range(0, len(all_series), chunk_size)):
    chunk_ids = all_series[i:i+chunk_size]
    chunk_data = df_forecast[df_forecast["unique_id"].isin(chunk_ids)]

    sf = StatsForecast(
        models=[AutoARIMA(season_length=12, seasonal=True)],
        freq="MS",
        n_jobs=4
    )

    forecast_chunk = sf.forecast(df=chunk_data, h=2)
    results.append(forecast_chunk)

# Concatenar resultados
forecast_all = pd.concat(results)
forecast_all.to_csv("autoarima_forecast.csv", index=False)
print("✅ Predicción completada. Archivo guardado como autoarima_forecast.csv")


📈 Ejecutando AutoARIMA por bloques...



 22%|██▏       | 58/263 [1:17:32<5:03:56, 88.96s/it]